## MonoT5

In [1]:
import json

with open('/kaggle/input/question-file-with-content-and-neg/updated_question_file_train_2negative.json', 'r') as f:
    question_data_train = json.load(f)
with open('/kaggle/input/question-file-with-content-and-neg/updated_question_file_test_2negative.json', 'r') as f:
    question_data_test = json.load(f)

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import numpy as np

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base")
model.to('cuda')

2024-07-05 10:50:17.945645: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 10:50:17.945755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 10:50:18.082265: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [3]:
# Create preprocess function
def preprocess_function(examples):
    model_inputs = tokenizer(examples["inputs"], max_length=1024, truncation=True, padding=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["labels"], max_length=10, truncation=True, padding=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Function to create training data with true, soft negative, and hard negative examples
def create_training_data(question_data):
    inputs = []
    labels = []
    for question in question_data:
        for relevant_law in question["relevant_laws"]:
            query = question["question"]
            document = relevant_law["content"]
            
            soft_negative = relevant_law["soft_negative"]
            hard_negative = relevant_law["hard_negative"]
            
            inputs.append(f"Câu hỏi: {query} Điều luật: {document} Có liên quan:")
            labels.append("đúng")
            
            for sn in soft_negative:
                inputs.append(f"Câu hỏi: {query} Điều luật: {sn} Có liên quan:")
                labels.append("sai")
            
            try: 
                for hn in hard_negative:
                    inputs.append(f"Câu hỏi: {query} Điều luật: {hn} Có liên quan:")
                    labels.append("sai")
            except:
                print(hard_negative)
    return inputs, labels

In [4]:
# Example usage with the question_data
inputs, labels = create_training_data(question_data_train + question_data_test)
dict_obj = {"inputs": inputs, "labels": labels}
dataset = Dataset.from_dict(dict_obj)
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=["inputs"], num_proc = 8)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=8):   0%|          | 0/102772 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword ar

In [5]:
len(inputs)

102772

In [6]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

# Training arguments
training_args = Seq2SeqTrainingArguments("tmp/",
      do_train=True,
      do_eval=False,
      num_train_epochs=2,
      learning_rate=2e-5,
#     warmup_ratio=0.05,
      weight_decay=0.01,
      per_device_train_batch_size=3,
#     per_device_eval_batch_size=16,
      logging_dir='./log',
      group_by_length=True,
      save_strategy="epoch",
      save_total_limit=3,
      #eval_steps=1,
      #evaluation_strategy="steps",
#     eval_strategy="epoch",
      fp16=True,
      report_to="none"  # Disable reporting to external tools
      )


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
500,1.008400
1000,0.229200
1500,0.192200
2000,0.178100
2500,0.167700
3000,0.175000
3500,0.160700
4000,0.155400
4500,0.179500
5000,0.144900


TrainOutput(global_step=68516, training_loss=0.11076512284673891, metrics={'train_runtime': 40845.6845, 'train_samples_per_second': 5.032, 'train_steps_per_second': 1.677, 'total_flos': 2.5033528131452928e+17, 'train_loss': 0.11076512284673891, 'epoch': 2.0})

In [7]:
# Save the model using pickle
import pickle

with open('vit5_to_monot5_full_16k_train.pkl', 'wb') as f:
    pickle.dump(model, f)